In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Lib.module.risk_budgeting import *
import student_mixture as sm
from tqdm import tqdm
import os
import sklearn as sl
import importlib

# Comparison of MDE across different number of assets

In [104]:
np.random.seed(0)

In [105]:
freq = 'B'
df_all = pd.concat([pd.read_excel('Data/SP_RC.xlsx',index_col=0, header=[0,1]), pd.read_excel('Data/SP_RC2.xlsx',index_col=0, header=[0,1])])
df_all = df_all.dropna(how='all').dropna(axis=1)
df_all = df_all.pct_change().dropna()
df_all = df_all.replace([np.inf, -np.inf], np.nan)
df_all = df_all[~df_all.index.duplicated(keep='first')]

## For 10 assets

#### Stochastic

In [106]:
first_level_index = df_all.columns.get_level_values(0)

In [107]:
selected_indices = np.random.choice(range(len(first_level_index)), 10, replace = False)
assets = first_level_index[selected_indices]
nb_asset = len(assets)
df = df_all[assets]
X = df.values
n_sm = 2
SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [3,6]).fit(X)
locs = SM_model.locations_
scales = SM_model.scales_
probs = SM_model.weights_
dofs = SM_model.dofs_
### Some adjustments on the parameters of the final model 

SM_model.locations_ = locs
SM_model.scales_ = scales
SM_model.weights_ = probs
SM_model.dofs_ = dofs

In [108]:
### Compute the Risk Budgeting portfolio under the above model

# ERC
budgets = np.ones(nb_asset)/nb_asset 
# Expected Shortfall alpha
alpha = .95 

SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)
gamma_star = optim_res.fun

print('==== Target Risk Bugdets ====')
print(np.round(budgets, 4))
print('==== $y^*$ ====')
print(np.round(optim_res.x, 4))

print('==== Risk Budgeting portfolio ($u^*$) ====')
print(np.round(SM_theta, 4))
print('==== VaR of the portfolio ====')
print(np.round(VaR_port, 4))
print('==== Expected Shortfall of the portfolio ====')
print(np.round(ES_port, 4))
print('==== Risk Contributions ====')
print(np.round(risk_contribs, 5))
print('==== Risk Contributions (normalized) ====')
print(np.round(risk_contribs/sum(risk_contribs), 2))

==== Target Risk Bugdets ====
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
==== $y^*$ ====
[4.9635 2.2469 2.3031 2.6274 3.4716 4.1732 2.6338 2.1152 3.7694 3.3774]
==== Risk Budgeting portfolio ($u^*$) ====
[0.1567 0.0709 0.0727 0.0829 0.1096 0.1317 0.0831 0.0668 0.119  0.1066]
==== VaR of the portfolio ====
0.02
==== Expected Shortfall of the portfolio ====
0.0316
==== Risk Contributions ====
[0.00316 0.00316 0.00316 0.00316 0.00316 0.00316 0.00316 0.00316 0.00316
 0.00316]
==== Risk Contributions (normalized) ====
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In [109]:
n_val=100000
X = SM_model.rvs(n_val)
np.random.shuffle(X)

In [110]:
M = 100

In [111]:
problem_10 = stoch_sol(X, M , n_val, nb_asset, budgets)

100%|██████████| 10/10 [00:24<00:00,  2.41s/it]


In [112]:
problem_10.results(SM_theta, real_ES= ES_port, real_VaR = VaR_port)

==== True Risk Budgeting portfolio ====
[0.1567 0.0709 0.0727 0.0829 0.1096 0.1317 0.0831 0.0668 0.119  0.1066]
==== MD Risk Budgeting portfolio ====
[0.1586 0.0699 0.0725 0.0833 0.1085 0.1331 0.0818 0.0674 0.1185 0.1063]
==== MD Risk Budgeting portfolio wo ====
[5.0807 2.2387 2.3242 2.6695 3.4766 4.2653 2.6199 2.1606 3.7978 3.407 ]
==== Expected Shortfall of the portfolio ====
0.0316
==== VaR of the portfolio ====
0.02
==== xi: VaR ====
0.0204
0.0316


In [113]:
dif_10 = abs((problem_10.y/sum(problem_10.y))- SM_theta)

In [114]:
round(np.mean(dif_10), 5)   

0.00087

#### Deterministic Case

In [115]:
M = 100
budgets = np.ones(nb_asset)/nb_asset
n_iter = 50000

In [116]:
sol_10 = determ_sol(SM_model, M, n_iter, nb_asset,  budgets)

100%|██████████| 49999/49999 [04:55<00:00, 169.23it/s]


In [117]:
sol_10.results(SM_theta)

==== True Risk Budgeting portfolio ====
[0.1567 0.0709 0.0727 0.0829 0.1096 0.1317 0.0831 0.0668 0.119  0.1066]
==== MD Risk Budgeting portfolio ====
[0.157  0.0709 0.0727 0.0829 0.1095 0.1317 0.0831 0.0667 0.119  0.1066]


In [118]:
difdet_10 = abs((sol_10.y/sum(sol_10.y))- SM_theta)
difdet_10

array([2.87873094e-04, 4.49998913e-05, 3.97542470e-05, 5.23142639e-05,
       4.15135598e-05, 9.71473236e-07, 4.39519248e-05, 4.12876316e-05,
       8.85443021e-06, 3.38774794e-05])

In [119]:
round(np.mean(difdet_10), 5)   

6e-05

#### Fixed stepsize

In [120]:
n_iter = 10000

In [121]:
sol_10_fixed = determ_sol(SM_model, M, n_iter, nb_asset,  budgets,fixed_stepsize=True )

100%|██████████| 9999/9999 [00:58<00:00, 169.76it/s]


In [122]:
sol_10_fixed.results(SM_theta)

==== True Risk Budgeting portfolio ====
[0.1567 0.0709 0.0727 0.0829 0.1096 0.1317 0.0831 0.0668 0.119  0.1066]
==== MD Risk Budgeting portfolio ====
[0.1567 0.0709 0.0727 0.0829 0.1096 0.1317 0.0831 0.0668 0.119  0.1066]


In [123]:
difdet_10_fixed = abs((sol_10_fixed.y/sum(sol_10_fixed.y))- SM_theta)
round(np.mean(difdet_10_fixed), 4)   

0.0

## For 50 assets


#### Stochastic

In [124]:
selected_indices = np.random.choice(range(len(first_level_index)), 50, replace = False)
assets = first_level_index[selected_indices]
nb_asset = len(assets)
df = df_all[assets]
X = df.values
n_sm = 2
SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [3,6]).fit(X)
locs = SM_model.locations_
scales = SM_model.scales_
probs = SM_model.weights_
dofs = SM_model.dofs_
### Some adjustments on the parameters of the final model 

SM_model.locations_ = locs
SM_model.scales_ = scales
SM_model.weights_ = probs
SM_model.dofs_ = dofs


In [125]:
### Compute the Risk Budgeting portfolio under the above model

# ERC
budgets = np.ones(nb_asset)/nb_asset 
# Expected Shortfall alpha
alpha = .95 

SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)
gamma_star = optim_res.fun

In [126]:
n_val=100000
X = SM_model.rvs(n_val)
np.random.shuffle(X)

In [127]:
M = 100

In [128]:
problem_50= stoch_sol_highdim(X, M , n_val, nb_asset, budgets)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:23<00:00,  2.36s/it]


In [129]:
problem_50.results(SM_theta)

==== True Risk Budgeting portfolio ====
[0.0295 0.0148 0.0146 0.0161 0.0226 0.0276 0.0173 0.0132 0.0228 0.0205
 0.0249 0.0134 0.0209 0.0246 0.0299 0.0225 0.0193 0.0382 0.0316 0.0246
 0.0159 0.0176 0.0197 0.0249 0.0169 0.0261 0.0209 0.0196 0.0128 0.0304
 0.0143 0.0201 0.0219 0.0167 0.0172 0.0158 0.018  0.0115 0.023  0.0199
 0.016  0.0151 0.0154 0.024  0.0119 0.0152 0.0202 0.0153 0.0199 0.0149]
==== MD Risk Budgeting portfolio ====
[0.03   0.0149 0.0145 0.0162 0.0224 0.027  0.0171 0.0131 0.0225 0.021
 0.0248 0.0131 0.0213 0.0246 0.0298 0.0228 0.0196 0.0376 0.0313 0.0238
 0.0164 0.0177 0.0193 0.0255 0.0167 0.0267 0.0214 0.0198 0.0128 0.0307
 0.0143 0.0197 0.0224 0.0169 0.0172 0.0161 0.0177 0.0115 0.0234 0.0203
 0.016  0.0151 0.0152 0.0234 0.0121 0.0152 0.0199 0.0149 0.0198 0.0146]
==== MD Risk Budgeting portfolio wo ====
[1.0011 0.4986 0.4842 0.5416 0.7481 0.9017 0.5719 0.438  0.7516 0.7028
 0.8275 0.4377 0.7124 0.8225 0.9944 0.7602 0.6543 1.2558 1.044  0.7948
 0.547  0.5909 0.6436 0.8516

In [130]:
dif_50 = abs((problem_50.y/sum(problem_50.y))- SM_theta)
dif_50

array([4.36327364e-04, 1.34398084e-04, 1.23039484e-04, 1.23561050e-04,
       2.34673125e-04, 5.95039689e-04, 1.99704941e-04, 1.01851085e-04,
       3.33445009e-04, 4.90570728e-04, 1.63089176e-04, 2.65683144e-04,
       4.00829382e-04, 3.80098620e-05, 9.59938709e-05, 2.31878328e-04,
       2.54405129e-04, 6.09521474e-04, 3.95200722e-04, 7.99650140e-04,
       5.12911457e-04, 6.86301606e-05, 4.44472228e-04, 5.75727335e-04,
       1.72434880e-04, 6.56776561e-04, 5.07863469e-04, 1.69337115e-04,
       2.32804628e-05, 3.46177060e-04, 6.44607967e-05, 3.15954182e-04,
       5.33999615e-04, 2.10443530e-04, 5.08008793e-05, 3.08874470e-04,
       3.33510078e-04, 4.60960967e-05, 3.11927380e-04, 4.05255460e-04,
       2.47607679e-05, 2.50391124e-05, 1.64694487e-04, 5.32383068e-04,
       1.55314313e-04, 2.51443442e-05, 3.25994015e-04, 4.06948525e-04,
       7.09886372e-05, 2.47326833e-04])

In [131]:
round(np.mean(dif_50), 5)   

0.00028

#### Deterministic

In [132]:
M = 100
budgets = np.ones(nb_asset)/nb_asset
n_iter = 50000

In [133]:
sol_50 = determ_sol(SM_model, M, n_iter, nb_asset,  budgets)

  0%|          | 0/49999 [00:00<?, ?it/s]

100%|██████████| 49999/49999 [04:47<00:00, 173.64it/s]


In [134]:
sol_50.results(SM_theta)

==== True Risk Budgeting portfolio ====
[0.0295 0.0148 0.0146 0.0161 0.0226 0.0276 0.0173 0.0132 0.0228 0.0205
 0.0249 0.0134 0.0209 0.0246 0.0299 0.0225 0.0193 0.0382 0.0316 0.0246
 0.0159 0.0176 0.0197 0.0249 0.0169 0.0261 0.0209 0.0196 0.0128 0.0304
 0.0143 0.0201 0.0219 0.0167 0.0172 0.0158 0.018  0.0115 0.023  0.0199
 0.016  0.0151 0.0154 0.024  0.0119 0.0152 0.0202 0.0153 0.0199 0.0149]
==== MD Risk Budgeting portfolio ====
[0.0316 0.014  0.014  0.0154 0.0217 0.0269 0.0168 0.0125 0.024  0.0204
 0.0258 0.0127 0.0215 0.0249 0.0306 0.023  0.0187 0.0452 0.0311 0.0249
 0.0153 0.0171 0.0197 0.026  0.0164 0.0273 0.0214 0.0198 0.0121 0.0305
 0.0135 0.0203 0.0228 0.0158 0.0164 0.0152 0.0175 0.0108 0.023  0.0197
 0.0155 0.0143 0.0146 0.0249 0.0112 0.0143 0.0201 0.0147 0.0201 0.0142]


In [135]:
difdet_50 = abs((sol_50.y/sum(sol_50.y))- SM_theta)
round(np.mean(difdet_50), 5)   

0.00076

Fixed stepsize

In [136]:
n_iter = 10000
sol_50_fixed = determ_sol(SM_model, M, n_iter, nb_asset,  budgets,fixed_stepsize=True )
sol_50_fixed.results(SM_theta)

  0%|          | 0/9999 [00:00<?, ?it/s]

100%|██████████| 9999/9999 [00:57<00:00, 174.61it/s]

==== True Risk Budgeting portfolio ====
[0.0295 0.0148 0.0146 0.0161 0.0226 0.0276 0.0173 0.0132 0.0228 0.0205
 0.0249 0.0134 0.0209 0.0246 0.0299 0.0225 0.0193 0.0382 0.0316 0.0246
 0.0159 0.0176 0.0197 0.0249 0.0169 0.0261 0.0209 0.0196 0.0128 0.0304
 0.0143 0.0201 0.0219 0.0167 0.0172 0.0158 0.018  0.0115 0.023  0.0199
 0.016  0.0151 0.0154 0.024  0.0119 0.0152 0.0202 0.0153 0.0199 0.0149]
==== MD Risk Budgeting portfolio ====
[0.0295 0.0148 0.0146 0.0161 0.0226 0.0276 0.0173 0.0132 0.0228 0.0205
 0.0249 0.0134 0.0209 0.0246 0.0299 0.0225 0.0193 0.0382 0.0316 0.0246
 0.0159 0.0176 0.0197 0.0249 0.0169 0.0261 0.0209 0.0196 0.0128 0.0304
 0.0143 0.0201 0.0219 0.0167 0.0172 0.0158 0.018  0.0115 0.023  0.0199
 0.016  0.0151 0.0154 0.024  0.0119 0.0152 0.0202 0.0153 0.0199 0.0149]


In [137]:
difdet_50_fixed = abs((sol_50_fixed.y/sum(sol_50_fixed.y))- SM_theta)
round(np.mean(difdet_50_fixed), 4)   

0.0

## For 100 assets

#### Stochastic

In [138]:
selected_indices = np.random.choice(range(len(first_level_index)), 100, replace = False)
assets = first_level_index[selected_indices]
nb_asset = len(assets)
df = df_all[assets]
X = df.values
n_sm = 2
SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [3,6]).fit(X)
locs = SM_model.locations_
scales = SM_model.scales_
probs = SM_model.weights_
dofs = SM_model.dofs_
### Some adjustments on the parameters of the final model 

SM_model.locations_ = locs
SM_model.scales_ = scales
SM_model.weights_ = probs
SM_model.dofs_ = dofs

In [139]:
### Compute the Risk Budgeting portfolio under the above model

# ERC
budgets = np.ones(nb_asset)/nb_asset 
# Expected Shortfall alpha
alpha = .95 

SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)
gamma_star = optim_res.fun

In [140]:
n_val=100000
X = SM_model.rvs(n_val)
np.random.shuffle(X)

In [141]:
M = 100

In [142]:
problem_100= stoch_sol_highdim(X, M , n_val, nb_asset, budgets)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:26<00:00,  2.68s/it]


In [143]:
problem_100.results(SM_theta)

==== True Risk Budgeting portfolio ====
[0.0149 0.0074 0.0073 0.0079 0.0114 0.014  0.0087 0.0066 0.0114 0.0102
 0.0123 0.0066 0.0104 0.0124 0.015  0.0115 0.0097 0.0191 0.0161 0.0125
 0.0078 0.0087 0.0097 0.0123 0.0084 0.0128 0.0105 0.0098 0.0063 0.0148
 0.007  0.01   0.0108 0.0083 0.0087 0.008  0.0092 0.0056 0.0117 0.0097
 0.0079 0.0075 0.0076 0.0119 0.0059 0.0077 0.0099 0.0075 0.01   0.0075
 0.0069 0.0103 0.0093 0.0092 0.0097 0.0105 0.0106 0.0102 0.008  0.0212
 0.0113 0.0085 0.0081 0.0077 0.0083 0.0087 0.0161 0.0098 0.0179 0.0069
 0.0062 0.0107 0.0097 0.007  0.017  0.0072 0.0086 0.0097 0.0107 0.0107
 0.0077 0.0097 0.0116 0.018  0.0101 0.0094 0.0095 0.007  0.0098 0.0069
 0.0064 0.007  0.0097 0.0113 0.0085 0.0155 0.0096 0.0099 0.0073 0.0103]
==== MD Risk Budgeting portfolio ====
[0.0152 0.0071 0.0072 0.0078 0.0112 0.0134 0.0085 0.0065 0.0122 0.0102
 0.0125 0.0063 0.0107 0.0126 0.0151 0.0113 0.0096 0.0195 0.0155 0.0122
 0.0077 0.0086 0.0097 0.0131 0.0081 0.0134 0.0104 0.0103 0.0061 0.014

In [144]:
dif_100 = abs((problem_100.y/sum(problem_100.y))- SM_theta)
dif_100

array([3.26149418e-04, 3.51400478e-04, 9.56935934e-05, 4.54621166e-05,
       2.10269856e-04, 5.83766551e-04, 2.08557923e-04, 8.71767320e-05,
       7.41309474e-04, 2.86012162e-05, 1.68854666e-04, 2.85704536e-04,
       2.25499200e-04, 2.02342267e-04, 9.83970683e-05, 1.90114141e-04,
       4.58542832e-05, 3.95781520e-04, 5.74823231e-04, 2.58977843e-04,
       1.42614208e-04, 5.81438314e-05, 8.18751452e-05, 7.86421685e-04,
       3.64494800e-04, 5.12831408e-04, 9.28482609e-05, 4.92548691e-04,
       1.72581632e-04, 6.47787327e-05, 1.54704398e-04, 1.36886501e-04,
       2.07530863e-04, 1.62908121e-04, 5.62162141e-04, 8.58034898e-05,
       8.25654232e-05, 2.34299655e-04, 1.22284433e-05, 3.81173723e-04,
       2.43869578e-05, 8.87606958e-05, 3.64351420e-05, 4.97211599e-04,
       2.90308388e-04, 5.69473120e-04, 4.78143954e-05, 6.76573051e-05,
       2.03871393e-04, 2.51272060e-04, 3.65564104e-04, 6.12072406e-04,
       1.72852474e-04, 8.42771108e-05, 2.91558305e-04, 4.42287360e-04,
      

In [145]:
round(np.mean(dif_100), 5)   

0.00025

#### Deterministic

In [146]:
M = 100
budgets = np.ones(nb_asset)/nb_asset
n_iter = 50000

In [147]:
sol_100 = determ_sol(SM_model, M, n_iter, nb_asset,  budgets)

  0%|          | 10/49999 [00:00<09:21, 89.01it/s]

100%|██████████| 49999/49999 [04:49<00:00, 172.54it/s]


In [148]:
sol_100.results(SM_theta)

==== True Risk Budgeting portfolio ====
[0.0149 0.0074 0.0073 0.0079 0.0114 0.014  0.0087 0.0066 0.0114 0.0102
 0.0123 0.0066 0.0104 0.0124 0.015  0.0115 0.0097 0.0191 0.0161 0.0125
 0.0078 0.0087 0.0097 0.0123 0.0084 0.0128 0.0105 0.0098 0.0063 0.0148
 0.007  0.01   0.0108 0.0083 0.0087 0.008  0.0092 0.0056 0.0117 0.0097
 0.0079 0.0075 0.0076 0.0119 0.0059 0.0077 0.0099 0.0075 0.01   0.0075
 0.0069 0.0103 0.0093 0.0092 0.0097 0.0105 0.0106 0.0102 0.008  0.0212
 0.0113 0.0085 0.0081 0.0077 0.0083 0.0087 0.0161 0.0098 0.0179 0.0069
 0.0062 0.0107 0.0097 0.007  0.017  0.0072 0.0086 0.0097 0.0107 0.0107
 0.0077 0.0097 0.0116 0.018  0.0101 0.0094 0.0095 0.007  0.0098 0.0069
 0.0064 0.007  0.0097 0.0113 0.0085 0.0155 0.0096 0.0099 0.0073 0.0103]
==== MD Risk Budgeting portfolio ====
[0.0169 0.0059 0.0068 0.0072 0.0107 0.0116 0.0082 0.0057 0.0133 0.0102
 0.0133 0.0056 0.0119 0.0131 0.0155 0.0114 0.0087 0.0246 0.0134 0.0125
 0.0074 0.0082 0.0105 0.0139 0.0077 0.0143 0.0118 0.0107 0.0053 0.013

In [149]:
difdet_100 = abs((sol_100.y/sum(sol_100.y))- SM_theta)
round(np.mean(difdet_100), 5)   

0.00115

Fixed stepsize

In [150]:
n_iter = 10000
sol_100_fixed = determ_sol(SM_model, M, n_iter, nb_asset,  budgets,fixed_stepsize=True )
sol_100_fixed.results(SM_theta)

  0%|          | 0/9999 [00:00<?, ?it/s]

100%|██████████| 9999/9999 [00:58<00:00, 171.88it/s]

==== True Risk Budgeting portfolio ====
[0.0149 0.0074 0.0073 0.0079 0.0114 0.014  0.0087 0.0066 0.0114 0.0102
 0.0123 0.0066 0.0104 0.0124 0.015  0.0115 0.0097 0.0191 0.0161 0.0125
 0.0078 0.0087 0.0097 0.0123 0.0084 0.0128 0.0105 0.0098 0.0063 0.0148
 0.007  0.01   0.0108 0.0083 0.0087 0.008  0.0092 0.0056 0.0117 0.0097
 0.0079 0.0075 0.0076 0.0119 0.0059 0.0077 0.0099 0.0075 0.01   0.0075
 0.0069 0.0103 0.0093 0.0092 0.0097 0.0105 0.0106 0.0102 0.008  0.0212
 0.0113 0.0085 0.0081 0.0077 0.0083 0.0087 0.0161 0.0098 0.0179 0.0069
 0.0062 0.0107 0.0097 0.007  0.017  0.0072 0.0086 0.0097 0.0107 0.0107
 0.0077 0.0097 0.0116 0.018  0.0101 0.0094 0.0095 0.007  0.0098 0.0069
 0.0064 0.007  0.0097 0.0113 0.0085 0.0155 0.0096 0.0099 0.0073 0.0103]
==== MD Risk Budgeting portfolio ====
[0.0149 0.0074 0.0073 0.0079 0.0114 0.014  0.0087 0.0066 0.0114 0.0102
 0.0123 0.0066 0.0104 0.0124 0.015  0.0115 0.0097 0.0191 0.0161 0.0125
 0.0078 0.0087 0.0097 0.0123 0.0084 0.0128 0.0105 0.0098 0.0063 0.014

In [151]:
difdet_100_fixed = abs((sol_100_fixed.y/sum(sol_100_fixed.y))- SM_theta)
round(np.mean(difdet_100_fixed), 4) 

0.0

## For 250 assets

#### Stochastic

In [152]:
selected_indices = np.random.choice(range(len(first_level_index)), 250, replace = False)
assets = first_level_index[selected_indices]
nb_asset = len(assets)
df = df_all[assets]
X = df.values
n_sm = 2
SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [3,6]).fit(X)
locs = SM_model.locations_
scales = SM_model.scales_
probs = SM_model.weights_
dofs = SM_model.dofs_
### Some adjustments on the parameters of the final model 

SM_model.locations_ = locs
SM_model.scales_ = scales
SM_model.weights_ = probs
SM_model.dofs_ = dofs

In [153]:
### Compute the Risk Budgeting portfolio under the above model

# ERC
budgets = np.ones(nb_asset)/nb_asset 
# Expected Shortfall alpha
alpha = .95 

SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)
gamma_star = optim_res.fun

In [154]:
n_val=100000
X = SM_model.rvs(n_val)
np.random.shuffle(X)

In [155]:
M = 100

In [156]:
problem_250= stoch_sol_highdim(X, M , n_val, nb_asset, budgets)

100%|██████████| 10/10 [00:35<00:00,  3.58s/it]


In [157]:
problem_250.results(SM_theta)

==== True Risk Budgeting portfolio ====
[0.0058 0.0029 0.0028 0.0033 0.0049 0.0069 0.0034 0.0025 0.0045 0.0044
 0.0051 0.0023 0.0042 0.0049 0.0056 0.0046 0.0039 0.0075 0.0071 0.0043
 0.0032 0.0036 0.0033 0.0048 0.0035 0.0055 0.0037 0.004  0.0024 0.0067
 0.0028 0.0041 0.0045 0.0033 0.0036 0.0032 0.0036 0.0023 0.0049 0.004
 0.0028 0.0031 0.0032 0.0047 0.0023 0.0032 0.0036 0.0027 0.0038 0.003
 0.0027 0.0043 0.0037 0.0037 0.004  0.0045 0.0039 0.0039 0.0034 0.0085
 0.0048 0.0035 0.0033 0.0028 0.0034 0.0036 0.0063 0.004  0.0074 0.0026
 0.0022 0.0043 0.004  0.0026 0.0072 0.003  0.0034 0.0039 0.0044 0.0043
 0.0031 0.004  0.0052 0.0082 0.0038 0.0037 0.0038 0.0027 0.0039 0.0029
 0.0024 0.0027 0.0034 0.0043 0.0035 0.006  0.0036 0.0034 0.003  0.0036
 0.0027 0.0032 0.0022 0.004  0.0031 0.0053 0.0058 0.0026 0.0042 0.0083
 0.0042 0.0037 0.0045 0.0031 0.004  0.0053 0.0036 0.0044 0.0025 0.0035
 0.0026 0.0036 0.0086 0.0044 0.0032 0.0037 0.0051 0.0036 0.0026 0.0038
 0.0064 0.0056 0.0027 0.0042 0.0026 0.0

In [158]:
dif_250 = abs((problem_250.y/sum(problem_250.y))- SM_theta)
dif_250

array([1.17836560e-04, 1.36872580e-04, 7.63971972e-05, 2.21623466e-04,
       5.89979653e-04, 1.61778674e-03, 6.71387441e-05, 1.60591414e-05,
       3.95227122e-04, 3.39475030e-04, 1.99963231e-04, 9.07549709e-05,
       5.40920696e-04, 1.51342206e-04, 3.75035142e-04, 9.57657745e-05,
       2.17146395e-04, 3.55918442e-04, 1.57035878e-03, 2.09823923e-04,
       2.76532803e-04, 9.64488517e-05, 2.66543086e-04, 1.18360648e-04,
       5.14967639e-05, 1.15276137e-04, 2.33933355e-04, 3.07815273e-04,
       6.46774948e-05, 9.20782346e-04, 1.36382249e-05, 1.45319238e-04,
       5.44521109e-04, 1.37657978e-04, 2.55325236e-04, 1.74030250e-04,
       9.84138002e-05, 2.00180527e-04, 3.71271960e-05, 1.46545784e-04,
       1.57150265e-04, 1.51380288e-04, 1.03831995e-04, 5.01903198e-05,
       6.30762304e-05, 9.14356929e-05, 1.84909333e-04, 8.94400061e-06,
       2.85752074e-04, 1.40227548e-05, 7.77526722e-05, 2.79564361e-04,
       4.12083972e-04, 1.52048851e-04, 6.73856936e-04, 1.93575207e-04,
      

In [159]:
round(np.mean(dif_250), 5)   

0.00026

#### Deterministic

In [160]:
M = 100
budgets = np.ones(nb_asset)/nb_asset
n_iter = 50000

In [161]:
sol_250 = determ_sol(SM_model, M, n_iter, nb_asset,  budgets)

100%|██████████| 49999/49999 [05:12<00:00, 160.13it/s]


In [162]:
sol_250.results(SM_theta)

==== True Risk Budgeting portfolio ====
[0.0058 0.0029 0.0028 0.0033 0.0049 0.0069 0.0034 0.0025 0.0045 0.0044
 0.0051 0.0023 0.0042 0.0049 0.0056 0.0046 0.0039 0.0075 0.0071 0.0043
 0.0032 0.0036 0.0033 0.0048 0.0035 0.0055 0.0037 0.004  0.0024 0.0067
 0.0028 0.0041 0.0045 0.0033 0.0036 0.0032 0.0036 0.0023 0.0049 0.004
 0.0028 0.0031 0.0032 0.0047 0.0023 0.0032 0.0036 0.0027 0.0038 0.003
 0.0027 0.0043 0.0037 0.0037 0.004  0.0045 0.0039 0.0039 0.0034 0.0085
 0.0048 0.0035 0.0033 0.0028 0.0034 0.0036 0.0063 0.004  0.0074 0.0026
 0.0022 0.0043 0.004  0.0026 0.0072 0.003  0.0034 0.0039 0.0044 0.0043
 0.0031 0.004  0.0052 0.0082 0.0038 0.0037 0.0038 0.0027 0.0039 0.0029
 0.0024 0.0027 0.0034 0.0043 0.0035 0.006  0.0036 0.0034 0.003  0.0036
 0.0027 0.0032 0.0022 0.004  0.0031 0.0053 0.0058 0.0026 0.0042 0.0083
 0.0042 0.0037 0.0045 0.0031 0.004  0.0053 0.0036 0.0044 0.0025 0.0035
 0.0026 0.0036 0.0086 0.0044 0.0032 0.0037 0.0051 0.0036 0.0026 0.0038
 0.0064 0.0056 0.0027 0.0042 0.0026 0.0

In [163]:
difdet_250 = abs((sol_250.y/sum(sol_250.y))- SM_theta)
round(np.mean(difdet_250), 5)   

0.00085

Fixed stpesize

In [164]:
n_iter = 10000
sol_250_fixed = determ_sol(SM_model, M, n_iter, nb_asset,  budgets,fixed_stepsize=True )
sol_250_fixed.results(SM_theta)

  0%|          | 0/9999 [00:00<?, ?it/s]

100%|██████████| 9999/9999 [01:00<00:00, 164.21it/s]

==== True Risk Budgeting portfolio ====
[0.0058 0.0029 0.0028 0.0033 0.0049 0.0069 0.0034 0.0025 0.0045 0.0044
 0.0051 0.0023 0.0042 0.0049 0.0056 0.0046 0.0039 0.0075 0.0071 0.0043
 0.0032 0.0036 0.0033 0.0048 0.0035 0.0055 0.0037 0.004  0.0024 0.0067
 0.0028 0.0041 0.0045 0.0033 0.0036 0.0032 0.0036 0.0023 0.0049 0.004
 0.0028 0.0031 0.0032 0.0047 0.0023 0.0032 0.0036 0.0027 0.0038 0.003
 0.0027 0.0043 0.0037 0.0037 0.004  0.0045 0.0039 0.0039 0.0034 0.0085
 0.0048 0.0035 0.0033 0.0028 0.0034 0.0036 0.0063 0.004  0.0074 0.0026
 0.0022 0.0043 0.004  0.0026 0.0072 0.003  0.0034 0.0039 0.0044 0.0043
 0.0031 0.004  0.0052 0.0082 0.0038 0.0037 0.0038 0.0027 0.0039 0.0029
 0.0024 0.0027 0.0034 0.0043 0.0035 0.006  0.0036 0.0034 0.003  0.0036
 0.0027 0.0032 0.0022 0.004  0.0031 0.0053 0.0058 0.0026 0.0042 0.0083
 0.0042 0.0037 0.0045 0.0031 0.004  0.0053 0.0036 0.0044 0.0025 0.0035
 0.0026 0.0036 0.0086 0.0044 0.0032 0.0037 0.0051 0.0036 0.0026 0.0038
 0.0064 0.0056 0.0027 0.0042 0.0026 0.0

In [165]:
difdet_250_fixed = abs((sol_250_fixed.y/sum(sol_250_fixed.y))- SM_theta)
round(np.mean(difdet_250_fixed), 4) 

0.0